In [1]:
!export TOKENIZERS_PARALLELISM=false
!rm -rf q_lora_korqa
!python -m pip install --upgrade pip
!pip install typing_extensions pydantic openai
!pip install datasets transformers peft trl bitsandbytes

In [2]:
# 학습 모델 로드 및 토큰 호출 한번더
from peft import PeftModel
from transformers import AutoModelForCausalLM,AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained("google/gemma-3-12b-it", torch_dtype="auto", device_map="cuda")
model = PeftModel.from_pretrained(base_model, "ohdyo/trip_qa_gem13b_v2")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-12b-it",add_bos=True)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.p

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
!pip install langchain
!pip install langchain-community

In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

memory_store = {}

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300
)


device = "cuda" if torch.cuda.is_available() else "cpu"
device

llm = HuggingFacePipeline(pipeline = pipe)

Device set to use cuda
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForC

In [5]:
memory = ConversationBufferMemory(memory_key="history", input_key="question", return_messages=False)

prompt_template = PromptTemplate(
    input_variables=["history", "question"],
    template="""
너는 항공기 반입 금지 물품 및 여행 정보를 전문적으로 알려주는 비서야.
하나의 질문에 대해서만 답변해줘. 다른 추가 질문은 하지 마.
아래는 지금까지의 대화야.

{history}
### 질문:
{question}

### 답변:"""
)


conversation_chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

/tmp/ipykernel_1735/3135108907.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", input_key="question", return_messages=False)
/tmp/ipykernel_1735/3135108907.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation_chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)


In [6]:
pip install flask --ignore-installed blinker

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)
Using cached MarkupSafe-3.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
from flask import Flask, request, jsonify
app = Flask(__name__)

In [8]:
@app.route("/refresh", methods=["GET"])
def refresh_memory():
    global memory, conversation_chain
    memory = ConversationBufferMemory(memory_key="history", input_key="question", return_messages=False)
    conversation_chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)
    return jsonify({"message": "Memory cleared successfully."})

In [10]:
@app.route('/input', methods=['POST'])
def handle_input():
    try:
        data = request.get_json()
        question = data.get("question", "")
        if not question:
            return jsonify({"error": "질문이 비어있습니다."}), 400

        result = conversation_chain.run(question=question)
        return jsonify({"question": question, "answer": result})
    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return jsonify({"error": str(e)}), 500

In [ ]:
app.run(host="0.0.0.0", port="8080")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.6.2:8080
Press CTRL+C to quit
/tmp/ipykernel_1735/3406964046.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain.run(question=question)
